In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=ffa530f70a0000e046f29e3a50e95e0ecc518822ca8609514e33204c09c8991a
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
from deepface import DeepFace
import cv2
import os
from datetime import datetime
from time import perf_counter
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

24-05-29 10:29:34 - Directory /root/.deepface created
24-05-29 10:29:34 - Directory /root/.deepface/weights created


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
image_path = "/content/gdrive/MyDrive/output_160/894016522_c720x720+0+0_r380x380!.jpeg"
video_path = "/content/gdrive/MyDrive/output_160/vvromanov_1.mp4"
video_name = os.path.splitext(os.path.basename(video_path))[0]

In [19]:
frames_folder_path = f"/content/gdrive/MyDrive/output_160/{video_name}_frames"
if not os.path.exists(frames_folder_path):
    os.makedirs(frames_folder_path)

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

n = 30
n_counter = 0
saved_frame_counter = 1
processed_frame_counter = 0
start_time = perf_counter()

while True:
  print(f"n_counter: {n_counter}")
  if n_counter == 0:
        ret, frame = cap.read()
        if ret:
            try:
                faces = DeepFace.extract_faces(
                    img_path=frame,
                    detector_backend="mtcnn"
                )

                print(f"faces detected: {len(faces)}")

                match_face = False
                for face in faces:
                  face_img = face["face"]

                  result = DeepFace.verify(
                        img1_path=image_path,
                        img2_path=face_img,
                        enforce_detection=False,
                        detector_backend="mtcnn",
                        model_name="VGG-Face",
                        distance_metric="euclidean_l2"
                    )

                  print(f"verification result: {result}")

                  if result["verified"]:
                        # Сохранение кадра, если лицо совпало
                        frame_save_path = os.path.join(frames_folder_path, f"{saved_frame_counter}.jpg")
                        cv2.imwrite(frame_save_path, frame)
                        saved_frame_counter += 1
                        match_face = True
                        break

                if not match_face:
                    print("No match face")

            except ValueError as e:
                # Если лицо на кадре не обнаружено
                print(f"No face found for current frame, error message: {str(e)}")
                n_counter = n // 2
                continue

            processed_frame_counter += 1

        else:
            print("Video ended")
            break

        n_counter = n
  else:
        n_counter -= 1
        _ = cap.grab()

time_elapsed = perf_counter() - start_time
print(f"Time elapsed: {time_elapsed}, FPS: {processed_frame_counter / time_elapsed}")
print(f"Frames processed: {processed_frame_counter}")

cap.release()